In [423]:
import numpy as np
import pandas as pd

# визуализация
import matplotlib.pyplot as plt
import seaborn as sns

# инструменты
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_recall_curve

# модели
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# скалер
from sklearn.preprocessing import StandardScaler

import warnings

In [164]:
df = pd.read_csv('[5]EPL_data_after_EDA.csv', index_col=0, parse_dates=['Date'])
df.head()

,Date,Time,Location,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AR,AvgH,AvgD,AvgA,Elo_HomeTeam,Elo_AwayTeam,Temperature,Humidity,Wind Speed,Condition
0,2024-04-25,20:00,EGKK,Brighton,Man City,0,4,A,0,3,...,0,6.568,4.890,1.426,1748.398682,2038.480591,44.750000,78.125000,5.687500,Fair
1,2024-04-24,20:00,EGCC,Man United,Sheffield United,4,2,H,1,1,...,0,1.304,6.056,8.154,1793.209839,1551.766602,44.979167,62.333333,7.062500,Mostly Cloudy
2,2024-04-24,20:00,EGGP,Everton,Liverpool,2,0,H,1,0,...,0,7.048,4.910,1.410,1692.962402,1923.581787,45.833333,65.523810,7.595238,Partly Cloudy
3,2024-04-24,20:00,EGLC,Crystal Palace,Newcastle,2,0,H,0,0,...,0,2.742,3.620,2.410,1708.831055,1811.576782,45.979167,59.250000,7.604167,Fair
4,2024-04-24,19:45,EGBB,Wolves,Bournemouth,0,1,A,0,1,...,1,2.698,3.586,2.476,1711.067993,1695.033447,43.958333,69.208333,6.520833,Fair


Перед тем как обучать модели, попробуем самостоятельно предсказать исходы матчей с помощью кастомных алгоритмов

При решении реальной задачи предсказания исхода игры большинства показателей, которые собраны в датафрейме, к сожалению, нет - статистика матча собирается только после того, как он пройдет. Тем не менее, возможно, модели можно научить предсказывать исход в течение матча по имеющейся уже статистике. Это можно применять в ставках в реальном времени матчей. Тогда эти данные не будут бесполезными

Одна из целей этого проекта - создать алгоритм, который будет хорошо предсказывать исход матча и зарабатывать как можно больше денег в букмекерской конторе при ставках на исход (Победа хозяев, Ничья или Победа гостей). Поэтому будем представлять, что на каждый матч делается ставка в 1 рубль по среднему коэффициенту букмекеров, уже подсчитанному в процессе предобработки данных.

Поэтому при поиске лучшей модели для предсказания результата матча будем ориентироваться на следующие метрики:
- accuracy
- ROI - выигрыш от ставки в букмекерской конторе (доля от вложенной суммы)

Их будет достаточно, потому что наша цель - спрогнозировать как можно больше правильных результатов и заработать как можно больше

## 1. Ставка на хозяев

В процессе разведочного анализа данных были обнаружены важные закономерности: команды, играющие на своем домашнем стадионе, статзначимо чаще выигрывают матч, чем гостевые команды. Попробуем сначала просто ставить на победу хозяев. Какими получатся результаты?

In [14]:
# возьмем нужные признаки из датафрейма

data_1 = df.loc[:, ['FTR', 'AvgH', 'AvgD', 'AvgA']]

# наш прогноз - домашняя команда всегда выигрывает
data_1['prognose'] = 'H'

# наша ставка = 1 рубль
data_1['bet'] = 1

# результат ставки (сумма выигрыша или проигрыша)
data_1['result'] = (data_1['prognose'] == data_1['FTR']) * data_1['AvgH'] * data_1['bet'] - (data_1['prognose'] != data_1['FTR']) * data_1['bet']

data_1.head()

,FTR,AvgH,AvgD,AvgA,prognose,bet,result
0,A,6.568,4.890,1.426,H,1,-1.000
1,H,1.304,6.056,8.154,H,1,1.304
2,H,7.048,4.910,1.410,H,1,7.048
3,H,2.742,3.620,2.410,H,1,2.742
4,A,2.698,3.586,2.476,H,1,-1.000


In [23]:
# доля угаданных исходов
accuracy_1 = (data_1['prognose'] == data_1['FTR']).sum() / data_1.shape[0]

# доход от вложений
roi_1 = data_1['result'].sum() / data_1.shape[0]

print(f'accuracy = {accuracy_1}')
print(f'ROI = {roi_1}')

accuracy = 0.444547134935305
ROI = 0.4153450400492914


Этот алгоритм не очень гибкий, зато дает 41.5% дохода от вложенных средств и угадывает 44.4% исходов матчей! Если бы еще можно было придумать как предсказывать ничьи, возможно результаты стали бы лучше

## 2. Ставка на Elo-рейтинг

Напомню, Elo-рейтинг отражает силу команды. Он пересчитывается после каждой игры. Чем более неожиданный результат происходит в игре команды, тем сильне он меняется (проигрыш - рейтинг падает, победа - растет)

Было выяснено, что команды с большим рейтингом побеждают чаще. Попробуем опираться только на рейтинг и ставить на ту команду, у которой он больше. Ничью предсказывать не будем

In [39]:
# возьмем нужные признаки из датафрейма

data_2 = df.loc[:, ['FTR', 'Elo_HomeTeam', 'Elo_AwayTeam','AvgH', 'AvgD', 'AvgA']]

# наш прогноз - домашняя команда всегда выiигрывает
data_2['prognose'] = data_2.apply(lambda row: 'H' if row['Elo_HomeTeam'] > row['Elo_AwayTeam'] else 'A', axis=1)

# наша ставка = 1 рубль
data_2['bet'] = 1

# результат ставки (сумма выигрыша или проигрыша)
data_2['result'] = data_2.apply(
                    lambda row: row['AvgH'] if (row['FTR'] == row['prognose'] and row['FTR'] == 'H') 
                            else row['AvgA'] if (row['FTR'] == row['prognose'] and row['FTR'] == 'A') 
                            else -1,
                    axis=1)

data_2.head()

,FTR,Elo_HomeTeam,Elo_AwayTeam,AvgH,AvgD,AvgA,prognose,bet,result
0,A,1748.398682,2038.480591,6.568,4.890,1.426,A,1,1.426
1,H,1793.209839,1551.766602,1.304,6.056,8.154,H,1,1.304
2,H,1692.962402,1923.581787,7.048,4.910,1.410,A,1,-1.000
3,H,1708.831055,1811.576782,2.742,3.620,2.410,A,1,-1.000
4,A,1711.067993,1695.033447,2.698,3.586,2.476,H,1,-1.000


In [40]:
# доля угаданных исходов
accuracy_2 = (data_2['prognose'] == data_2['FTR']).sum() / data_2.shape[0]

# доход от вложений
roi_2 = data_2['result'].sum() / data_2.shape[0]

print(f'accuracy = {accuracy_2}')
print(f'ROI = {roi_2}')

accuracy = 0.5402033271719039
ROI = 0.5200597658656809


Здесь результаты еще лучше! Угаданы 54% исходов и заработано 52% от вложений

## 3. Ставка на букмекеров

Чем ниже коэффициент в букмекерской конторе на какой-то исход, тем больше контора уверена в том, что произойдет этот исход. Попробуем использовать аналитические ресурсы букмекеров в свою пользу и будем выбирать исход, на который будет наименьший коэффициент

In [62]:
# возьмем нужные признаки из датафрейма

data_3 = df.loc[:, ['FTR','AvgH', 'AvgD', 'AvgA']]

# наш прогноз - домашняя команда всегда выiигрывает
data_3['prognose'] = data_3.apply(lambda row: row.index[np.argmin(row[1:]) + 1][-1],
                                 axis=1)

# наша ставка = 1 рубль
data_3['bet'] = 1

# результат ставки (сумма выигрыша или проигрыша)
data_3['result'] = data_3.apply(
                    lambda row: row['AvgH'] if (row['FTR'] == row['prognose'] and row['FTR'] == 'H') 
                            else row['AvgA'] if (row['FTR'] == row['prognose'] and row['FTR'] == 'A') 
                            else row['AvgD'] if (row['FTR'] == row['prognose'] and row['FTR'] == 'D') 
                            else -1,
                    axis=1)

data_3.head()

,FTR,AvgH,AvgD,AvgA,prognose,bet,result
0,A,6.568,4.890,1.426,A,1,1.426
1,H,1.304,6.056,8.154,H,1,1.304
2,H,7.048,4.910,1.410,A,1,-1.000
3,H,2.742,3.620,2.410,A,1,-1.000
4,A,2.698,3.586,2.476,A,1,2.476


In [63]:
# доля угаданных исходов
accuracy_3 = (data_3['prognose'] == data_3['FTR']).sum() / data_3.shape[0]

# доход от вложений
roi_3 = data_3['result'].sum() / data_3.shape[0]

print(f'accuracy = {accuracy_3}')
print(f'ROI = {roi_3}')

accuracy = 0.5577634011090573
ROI = 0.5347828096118299


Результаты стали еще лучше! Кстати, букмекеры никогда полностью не уверены в ничьей, поэтому данная модель тоже предсказывает только победы:

In [66]:
data_3['prognose'].value_counts()

prognose
H    1353
A     811
Name: count, dtype: int64

### Последняя предобработка перед машинным обучением

- Уберем признаки, которые не будут важными: Дата (не определяет исход матча), Название клубов (их силу все равно будет отражать Elo-рейтинг), количество голов: по ним определяется исход, поэтому их тоже убираем
- Отмасштабируем количественные признаки
- Преобразуем категориальные признаки, чтобы предотвратить появление большого количества после кодирования
- Закодируем категориальные признаки


__Убираем лишние__

In [356]:
df = df.loc[:, df.columns[~df.columns.isin(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'HTR'])]]
df

,Time,Location,FTR,Referee,HS,AS,HST,AST,HF,AF,...,AR,AvgH,AvgD,AvgA,Elo_HomeTeam,Elo_AwayTeam,Temperature,Humidity,Wind Speed,Condition
0,20:00,EGKK,A,J Gillett,7,14,3,6,10,3,...,0,6.568000,4.890000,1.426000,1748.398682,2038.480591,44.750000,78.125000,5.687500,Fair
1,20:00,EGCC,H,M Salisbury,25,10,13,4,7,9,...,0,1.304000,6.056000,8.154000,1793.209839,1551.766602,44.979167,62.333333,7.062500,Mostly Cloudy
2,20:00,EGGP,H,A Madley,16,23,6,7,6,13,...,0,7.048000,4.910000,1.410000,1692.962402,1923.581787,45.833333,65.523810,7.595238,Partly Cloudy
3,20:00,EGLC,H,T Bramall,20,7,7,2,10,15,...,0,2.742000,3.620000,2.410000,1708.831055,1811.576782,45.979167,59.250000,7.604167,Fair
4,19:45,EGBB,A,S Attwell,15,21,4,6,10,17,...,1,2.698000,3.586000,2.476000,1711.067993,1695.033447,43.958333,69.208333,6.520833,Fair
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,15:00,EGNT,A,M Atkinson,15,15,2,5,11,12,...,0,3.821667,3.420000,2.053333,1670.871338,1914.848877,57.444444,72.138889,5.916667,Fair
2160,15:00,EGNM,A,C Kavanagh,6,13,1,4,9,8,...,0,6.276667,3.970000,1.590000,1567.101318,1837.004272,58.157895,73.526316,8.842105,Fair
2161,15:00,EGLC,A,M Dean,15,10,6,9,9,11,...,0,2.466667,3.360000,2.950000,1633.799683,1692.951660,61.973684,69.868421,8.078947,Fair
2162,15:00,EGHH,H,K Friend,12,10,4,1,11,9,...,0,1.895000,3.538333,4.388333,1673.780518,1576.490356,60.296296,88.000000,7.851852,Mostly Cloudy


- Масштабируем. Будем нормализовать значения, потому что в некоторых признаках могут встретиться выбросы и MinMaxScaler не подойдет

In [357]:
scaler = StandardScaler()

# разделяем на категориальные и численные значения
df_obj = df[df.columns[df.dtypes == 'object']]
df_num = df[df.columns[df.dtypes != 'object']]

df_num_scaled = pd.DataFrame(scaler.fit_transform(df_num), columns = df_num.columns)
df_num_scaled.head()

,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,AvgH,AvgD,AvgA,Elo_HomeTeam,Elo_AwayTeam,Temperature,Humidity,Wind Speed
0,-1.192303,0.487106,-0.683758,0.790516,-0.153877,-2.115588,-0.563377,-0.243831,0.302515,-1.396759,-0.229768,-0.249364,1.488479,0.446627,-0.741217,-0.130670,2.331726,-0.618948,-0.109709,-0.800174
1,1.928419,-0.296755,3.124872,-0.037298,-1.025575,-0.463230,1.045847,-0.243831,-1.294176,-0.613503,-0.229768,-0.249364,-0.719612,1.268616,0.767687,0.249381,-1.799032,-0.592628,-1.638698,-0.486735
2,0.368058,2.250795,0.458831,1.204422,-1.316141,0.638342,-0.563377,2.991077,-1.294176,0.953010,-0.229768,-0.249364,1.689824,0.460727,-0.744805,-0.600834,1.356576,-0.494527,-1.329788,-0.365294
3,1.061552,-0.884652,0.839694,-0.865111,-0.153877,1.189128,0.402157,-0.962699,1.100861,0.953010,-0.229768,-0.249364,-0.116414,-0.448678,-0.520533,-0.466250,0.405985,-0.477778,-1.937234,-0.363259
4,0.194685,1.858865,-0.302895,0.790516,-0.153877,1.739914,0.724002,2.631642,1.899206,-0.613503,-0.229768,3.718454,-0.134871,-0.472646,-0.505731,-0.447278,-0.583122,-0.709871,-0.973043,-0.610211


In [358]:
data = pd.concat([df_obj, df_num_scaled], axis=1)
data.head()

,Time,Location,FTR,Referee,Condition,HS,AS,HST,AST,HF,...,HR,AR,AvgH,AvgD,AvgA,Elo_HomeTeam,Elo_AwayTeam,Temperature,Humidity,Wind Speed
0,20:00,EGKK,A,J Gillett,Fair,-1.192303,0.487106,-0.683758,0.790516,-0.153877,...,-0.229768,-0.249364,1.488479,0.446627,-0.741217,-0.130670,2.331726,-0.618948,-0.109709,-0.800174
1,20:00,EGCC,H,M Salisbury,Mostly Cloudy,1.928419,-0.296755,3.124872,-0.037298,-1.025575,...,-0.229768,-0.249364,-0.719612,1.268616,0.767687,0.249381,-1.799032,-0.592628,-1.638698,-0.486735
2,20:00,EGGP,H,A Madley,Partly Cloudy,0.368058,2.250795,0.458831,1.204422,-1.316141,...,-0.229768,-0.249364,1.689824,0.460727,-0.744805,-0.600834,1.356576,-0.494527,-1.329788,-0.365294
3,20:00,EGLC,H,T Bramall,Fair,1.061552,-0.884652,0.839694,-0.865111,-0.153877,...,-0.229768,-0.249364,-0.116414,-0.448678,-0.520533,-0.466250,0.405985,-0.477778,-1.937234,-0.363259
4,19:45,EGBB,A,S Attwell,Fair,0.194685,1.858865,-0.302895,0.790516,-0.153877,...,-0.229768,3.718454,-0.134871,-0.472646,-0.505731,-0.447278,-0.583122,-0.709871,-0.973043,-0.610211


__Преобразуем категориальные:__

- Время (поделим на ранний день, день и вечер)
- Судьи. Если работал на маленьком количестве игр, то 'Другой'

In [359]:
# преобразуем время
data['Time'] = data['Time'].apply(lambda x: 'Early day' if x <= '14:00'
                        else 'Day' if '14:00' < x <= '18:00'
                        else 'Evening')

# преобразуем судей
data['Referee'] = data['Referee'].apply(lambda x: 'Other' if df['Referee'].value_counts()[x] < 10
                                               else x)

__Кодируем категориальные признаки__
И делим на признаки и таргет

In [360]:
X = data[data.columns[data.columns != 'FTR']]  # признаки 
y = data['FTR']  # таргет

X = pd.get_dummies(X, dtype=int)
X.head()

,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,...,Condition_Fair,Condition_Fog,Condition_Light Rain,Condition_Mist,Condition_Mostly Cloudy,Condition_Partly Cloudy,Condition_Rain,Condition_Showers in the Vicinity,Condition_Windy,Condition_Winter
0,-1.192303,0.487106,-0.683758,0.790516,-0.153877,-2.115588,-0.563377,-0.243831,0.302515,-1.396759,...,1,0,0,0,0,0,0,0,0,0
1,1.928419,-0.296755,3.124872,-0.037298,-1.025575,-0.463230,1.045847,-0.243831,-1.294176,-0.613503,...,0,0,0,0,1,0,0,0,0,0
2,0.368058,2.250795,0.458831,1.204422,-1.316141,0.638342,-0.563377,2.991077,-1.294176,0.953010,...,0,0,0,0,0,1,0,0,0,0
3,1.061552,-0.884652,0.839694,-0.865111,-0.153877,1.189128,0.402157,-0.962699,1.100861,0.953010,...,1,0,0,0,0,0,0,0,0,0
4,0.194685,1.858865,-0.302895,0.790516,-0.153877,1.739914,0.724002,2.631642,1.899206,-0.613503,...,1,0,0,0,0,0,0,0,0,0


- Делим данные на тренировочную и тестовую выборки
- Прописываем stratify по таргету, потому что таргет не сбалансирован - побед хозяев намного больше

In [361]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

Достанем коэффициенты, чтобы потом считать по ним ROI

In [362]:
books = df[['AvgH', 'AvgD', 'AvgA']]
books.head()

,AvgH,AvgD,AvgA
0,6.568,4.890,1.426
1,1.304,6.056,8.154
2,7.048,4.910,1.410
3,2.742,3.620,2.410
4,2.698,3.586,2.476


## 4. KNN

Приступаем к машинному обучению. Пока что будем использовать все данные о матчах. При решении реальной задачи нам будут доступны только коэффициенты букмекеров, Elo-рейтинг и погода. С этим разберемся позже

С помощью кросс-валидации найдем оптимальный гиперпараметр k - число соседей

In [363]:
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': np.arange(1, 31)}

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_k = grid_search.best_params_['n_neighbors']
best_k

25

In [364]:
# обучаем модель и получаем предикт

knn_best = KNeighborsClassifier(best_k)

knn_best.fit(X_train, y_train)

y_pred = knn_best.predict(X_test)

In [365]:
# функция для получения таблицы с коэффициентами и результатом ставки

def books_table(y_pred, y_true):
    # кодируем результат ставки: верно поставили = 1, неверно = 0
    bet_result = (y_true == y_pred).astype(int)
    bet_result.rename('bet_result', inplace=True)

    books_knn = pd.concat([books.iloc[y_true.index], y_true, bet_result], axis=1)

    books_knn['return'] = books_knn.apply(
                        lambda row: row['AvgH'] if (row['bet_result'] == 1 and row['FTR'] == 'H') 
                                else row['AvgA'] if (row['bet_result'] == 1 and row['FTR'] == 'A') 
                                else row['AvgD'] if (row['bet_result'] == 1 and row['FTR'] == 'D') 
                                else -1,
                        axis=1)

    return books_knn

In [366]:
books_knn_test = books_table(y_pred, y_test)
books_knn_train = books_table(knn_best.predict(X_train), y_train)

books_knn_test

,AvgH,AvgD,AvgA,FTR,bet_result,return
215,2.130000,3.578333,3.285000,A,1,3.285000
1214,3.860000,3.516667,1.983333,H,0,-1.000000
2040,2.800000,3.466667,2.558333,H,1,2.800000
1602,3.490000,3.845000,1.995000,D,0,-1.000000
797,2.278333,3.596667,2.975000,H,1,2.278333
...,...,...,...,...,...,...
1934,20.251667,8.180000,1.146667,A,1,1.146667
1833,2.615000,3.170000,2.945000,H,1,2.615000
10,2.734000,3.588000,2.428000,A,1,2.428000
1435,2.625000,3.426667,2.681667,H,1,2.625000


In [367]:
accuracy_4_test = accuracy_score(y_test, y_pred)
accuracy_4_train = accuracy_score(y_train, knn_best.predict(X_train))

roi_4_test = books_knn_test['return'].sum() / books_knn_test['return'].shape[0]
roi_4_train = books_knn_train['return'].sum() / books_knn_train['return'].shape[0]

print(f'accuracy on test = {accuracy_4_test}')
print(f'accuracy on train = {accuracy_4_train}\n')

print(f'ROI on test = {roi_4_test}')
print(f'ROI on train = {roi_4_train}')

accuracy on test = 0.6
accuracy on train = 0.6413474240422721

ROI on test = 0.8781671794871794
ROI on train = 0.9923494055482166


Модель умеет зарабатывать в два раза больше вложений в ставки! accuracy выросла, но есть куда расти еще.

__Отличные результаты!__ Лучше, чем наивные предсказания без машинного обучения. К тому же, теперь модель умеет предсказывать ничьи.

## 5. Логистическая регрессия

In [368]:
warnings.filterwarnings('ignore')

logreg = LogisticRegression(multi_class='multinomial')
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

books_logreg_test = books_table(y_pred, y_test)
books_logreg_train = books_table(logreg.predict(X_train), y_train)

accuracy_5_test = accuracy_score(y_test, y_pred)
accuracy_5_train = accuracy_score(y_train, logreg.predict(X_train))

roi_5_test = books_logreg_test['return'].sum() / books_logreg_test['return'].shape[0]
roi_5_train = books_logreg_train['return'].sum() / books_logreg_train['return'].shape[0]

print(f'accuracy on test = {accuracy_5_test}')
print(f'accuracy on train = {accuracy_5_train}\n')

print(f'ROI on test = {roi_5_test}')
print(f'ROI on train = {roi_5_train}')

accuracy on test = 0.62
accuracy on train = 0.6571994715984147

ROI on test = 0.9995764102564102
ROI on train = 1.0956508146191106


__Вывод:__ Стало еще лучше! ROI вырос, accuracy тоже немного выросла!

Посмотрим еще на важность признаков для будущего исследования:

In [369]:
logreg_A_coefs = pd.Series(data=logreg.coef_[0], index=logreg.feature_names_in_)
logreg_D_coefs = pd.Series(data=logreg.coef_[1], index=logreg.feature_names_in_)
logreg_H_coefs = pd.Series(data=logreg.coef_[2], index=logreg.feature_names_in_)

print(f'H: Самые важные признаки со знаком -:\n {logreg_H_coefs.sort_values()[:5]}\n')
print(f'H: Самые важные признаки со знаком +:\n {logreg_H_coefs.sort_values()[-5:]}\n')

print(f'A: Самые важные признаки со знаком -:\n {logreg_A_coefs.sort_values()[:5]}\n')
print(f'A: Самые важные признаки со знаком +:\n {logreg_A_coefs.sort_values()[-5:]}\n')

print(f'D: Самые важные признаки со знаком -:\n {logreg_D_coefs.sort_values()[:5]}\n')
print(f'D: Самые важные признаки со знаком +:\n {logreg_D_coefs.sort_values()[-5:]}\n')

H: Самые важные признаки со знаком -:
 AST                    -0.692991
AvgH                   -0.607047
Location_EGHI          -0.576493
Referee_T Harrington   -0.437505
Referee_Other          -0.416189
dtype: float64

H: Самые важные признаки со знаком +:
 Location_EGGW          0.481303
Referee_S Barrott      0.501614
Referee_M Salisbury    0.524982
Referee_T Robinson     0.581301
HST                    0.835969
dtype: float64

A: Самые важные признаки со знаком -:
 HST                 -0.664669
Location_EGNT       -0.520287
Condition_Winter    -0.512127
Condition_Windy     -0.507528
Referee_T Bramall   -0.470011
dtype: float64

A: Самые важные признаки со знаком +:
 Referee_M Oliver    0.310353
AvgH                0.474236
Location_EGNX       0.488693
Condition_Fog       0.490592
AST                 0.791642
dtype: float64

D: Самые важные признаки со знаком -:
 Location_EGGW        -0.731315
Referee_S Barrott    -0.583952
Referee_T Robinson   -0.564757
Location_EGFF        -0.3822

Видно, что на предсказание сильно влияют имена судей и локация. Также важным параметром является коэффициент у букмекеров. Из игровых показателей важно количество ударов по воротам

## 6. Random Forest

In [401]:
# найдем оптимальные значения гиперпараметров с помощью кросс-валидации

param_grid = {
    'n_estimators': [5, 8, 10, 12, 15, 20],
    'max_depth': np.arange(10, 51, 2),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 5, 10]}
    
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)

rf_best = grid_search.best_estimator_

In [402]:
rf_best

RandomForestClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=5,
                       n_estimators=15, random_state=42)

In [403]:
# оптимальная модель в действии

y_pred = rf_best.predict(X_test)

books_rf_test = books_table(y_pred, y_test)
books_rf_train = books_table(rf_best.predict(X_train), y_train)

accuracy_6_test = accuracy_score(y_test, y_pred)
accuracy_6_train = accuracy_score(y_train, rf_best.predict(X_train))

roi_6_test = books_rf_test['return'].sum() / books_rf_test['return'].shape[0]
roi_6_train = books_rf_train['return'].sum() / books_rf_train['return'].shape[0]

print(f'accuracy on test = {accuracy_6_test}')
print(f'accuracy on train = {accuracy_6_train}\n')

print(f'ROI on test = {roi_6_test}')
print(f'ROI on train = {roi_6_train}')

accuracy on test = 0.563076923076923
accuracy on train = 0.8190224570673712

ROI on test = 0.6743405128205128
ROI on train = 1.7893670189343902


In [412]:
pd.DataFrame(rf_best.feature_importances_, index=rf_best.feature_names_in_).sort_values(by=0)

,0
Condition_Winter,0.000000
Condition_Rain,0.000000
Location_EGGW,0.000000
Referee_J Gillett,0.000000
Referee_R East,0.000000
...,...
Elo_AwayTeam,0.057621
Elo_HomeTeam,0.075772
AST,0.089724
HST,0.094077


Самыми важными признаками оказались Elo-рейтинги, удары по воротам и коэффициент на победу гостей

Но это все работает на данных, которые нельзя получить до матча. Попробуем создать реальную модель, которая будет предсказывать исход только по известным показателям

## Урезаем данные до доступных только перед матчем

In [394]:
X_train_cut = X_train.loc[:, 
            X_train.columns[~X_train.columns.isin(['HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR'])]]

X_test_cut = X_test.loc[:, 
            X_train.columns[~X_train.columns.isin(['HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR'])]]

### KNN

In [396]:
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': np.arange(1, 31)}

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_k = grid_search.best_params_['n_neighbors']

# обучаем модель и получаем предикт

knn_best = KNeighborsClassifier(best_k)

knn_best.fit(X_train_cut, y_train)

y_pred = knn_best.predict(X_test_cut)

books_knn_test = books_table(y_pred, y_test)
books_knn_train = books_table(knn_best.predict(X_train_cut), y_train)

accuracy_4_test = accuracy_score(y_test, y_pred)
accuracy_4_train = accuracy_score(y_train, knn_best.predict(X_train_cut))

roi_4_test = books_knn_test['return'].sum() / books_knn_test['return'].shape[0]
roi_4_train = books_knn_train['return'].sum() / books_knn_train['return'].shape[0]

print(f'accuracy on test = {accuracy_4_test}')
print(f'accuracy on train = {accuracy_4_train}\n')

print(f'ROI on test = {roi_4_test}')
print(f'ROI on train = {roi_4_train}')

accuracy on test = 0.5276923076923077
accuracy on train = 0.583223249669749

ROI on test = 0.5545774358974359
ROI on train = 0.6973469837076177


### Логистическая регрессия

In [426]:
warnings.filterwarnings('ignore')

logreg = LogisticRegression(multi_class='multinomial')
logreg.fit(X_train_cut, y_train)

y_pred = logreg.predict(X_test_cut)

books_logreg_test = books_table(y_pred, y_test)
books_logreg_train = books_table(logreg.predict(X_train_cut), y_train)

accuracy_5_test = accuracy_score(y_test, y_pred)
accuracy_5_train = accuracy_score(y_train, logreg.predict(X_train_cut))

roi_5_test = books_logreg_test['return'].sum() / books_logreg_test['return'].shape[0]
roi_5_train = books_logreg_train['return'].sum() / books_logreg_train['return'].shape[0]

print(f'accuracy on test = {accuracy_5_test}')
print(f'accuracy on train = {accuracy_5_train}\n')

print(f'ROI on test = {roi_5_test}')
print(f'ROI on train = {roi_5_train}\n')

logreg_A_coefs = pd.Series(data=logreg.coef_[0], index=logreg.feature_names_in_)
logreg_D_coefs = pd.Series(data=logreg.coef_[1], index=logreg.feature_names_in_)
logreg_H_coefs = pd.Series(data=logreg.coef_[2], index=logreg.feature_names_in_)

print(f'H: Самые важные признаки со знаком -:\n {logreg_H_coefs.sort_values()[:5]}\n')
print(f'H: Самые важные признаки со знаком +:\n {logreg_H_coefs.sort_values()[-1:-6:-1]}\n')

print(f'A: Самые важные признаки со знаком -:\n {logreg_A_coefs.sort_values()[:5]}\n')
print(f'A: Самые важные признаки со знаком +:\n {logreg_A_coefs.sort_values()[-1:-6:-1]}\n')

print(f'D: Самые важные признаки со знаком -:\n {logreg_D_coefs.sort_values()[:5]}\n')
print(f'D: Самые важные признаки со знаком +:\n {logreg_D_coefs.sort_values()[-1:-6:-1]}\n')

accuracy on test = 0.5292307692307693
accuracy on train = 0.5752972258916776

ROI on test = 0.5142
ROI on train = 0.6402853368560105

H: Самые важные признаки со знаком -:
 AvgH                   -0.628668
Referee_Other          -0.553022
Location_EGHI          -0.396028
Referee_T Harrington   -0.386152
Referee_J Gillett      -0.315418
dtype: float64

H: Самые важные признаки со знаком +:
 Referee_S Barrott     0.583794
Referee_T Robinson    0.453381
Referee_R East        0.395351
Location_EGGW         0.368802
AvgD                  0.299675
dtype: float64

A: Самые важные признаки со знаком -:
 Condition_Winter    -0.687665
Referee_P Bankes    -0.576650
Referee_T Bramall   -0.565119
Location_EGNT       -0.537801
Condition_Windy     -0.515526
dtype: float64

A: Самые важные признаки со знаком +:
 AvgH             0.479543
Location_EGGW    0.313047
Condition_Fog    0.313035
Location_EGCN    0.288412
Location_EGNX    0.282631
dtype: float64

D: Самые важные признаки со знаком -:
 Locatio

### Random Forest

In [413]:
# найдем оптимальные значения гиперпараметров с помощью кросс-валидации

param_grid = {
    'n_estimators': [5, 8, 10, 12, 15, 20],
    'max_depth': np.arange(10, 51, 2),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 5, 10]}
    
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train_cut, y_train)

rf_best = grid_search.best_estimator_

In [415]:
# оптимальная модель в действии

y_pred = rf_best.predict(X_test_cut)

books_rf_test = books_table(y_pred, y_test)
books_rf_train = books_table(rf_best.predict(X_train_cut), y_train)

accuracy_6_test = accuracy_score(y_test, y_pred)
accuracy_6_train = accuracy_score(y_train, rf_best.predict(X_train_cut))

roi_6_test = books_rf_test['return'].sum() / books_rf_test['return'].shape[0]
roi_6_train = books_rf_train['return'].sum() / books_rf_train['return'].shape[0]

print(f'accuracy on test = {accuracy_6_test}')
print(f'accuracy on train = {accuracy_6_train}\n')

print(f'ROI on test = {roi_6_test}')
print(f'ROI on train = {roi_6_train}')

accuracy on test = 0.5492307692307692
accuracy on train = 0.6188903566710701

ROI on test = 0.5590923076923077
ROI on train = 0.7890907089387935


In [421]:
pd.Series(rf_best.feature_importances_, index=rf_best.feature_names_in_).sort_values()[-1:-10:-1]

AvgA             0.213492
AvgH             0.188166
Elo_AwayTeam     0.145537
AvgD             0.101855
Elo_HomeTeam     0.092615
Humidity         0.051438
Wind Speed       0.047723
Temperature      0.045126
Location_EGCC    0.013073
dtype: float64

Самые важные параметры - именно те, на основе которых мы и делали наивные прогнозы в начале тетрадки. Также важными оказались Влажность, Скорость ветра и Температура

Кажется, что на этих данных Random Forest - более разумная модель, чем ЛогРег, потому что важные признаки в RF намного более логично подобраны

## Итоги

Посмотрим на итоговые метрики всех представленных моделей. Результаты моделей машинного обучения будут учитываться только на срезанных данных (только тех, которые можно иметь до матча) и отображаться по тестовым данным

In [422]:
pd.DataFrame({'Модель': ['Ставка на хозяев', 'Ставка на Elo-рейтинг', 'Ставка на букмекеров', 'KNN', 'LogReg', 'RandomForest'],
              'accuracy': [accuracy_1, accuracy_2, accuracy_3, accuracy_4_test, accuracy_5_test, accuracy_6_test],
              'ROI': [roi_1, roi_2, roi_3, roi_4_test, roi_5_test, roi_6_test]})

,Модель,accuracy,ROI
0,Ставка на хозяев,0.444547,0.415345
1,Ставка на Elo-рейтинг,0.540203,0.520060
2,Ставка на букмекеров,0.557763,0.534783
3,KNN,0.527692,0.554577
4,LogReg,0.529231,0.514200
5,RandomForest,0.549231,0.559092


__Выводы:__
- Наибольшую выгоду можно получить по модели Random Forest. Итоговая ROI вышла 55.9%, а accuracy -- 54.9%
- Наибольшую точность можно получить не с помощью ML, а по алгоритму Ставки на букмекеров (accuracy = 55.7%)

Интересно, что наибольшая accuracy вышла не у RF, а у алгоритма Ставки на букмекеров. Но почему ROI у этой стратегии ниже? Потому что в ней мы всегда выбираем наименьший букмекерский коэффициент. Соответственно, хотя больше исходов удается предугадать, большую награду за один матч не получить. В то же время, RF может не соглашаться с букмекерами и выбирать менее вероятные исходы, за которые можно получить бОльший выигрыш.

__Конечно,__ этот проект можно развивать и дальше:
- Для более точного и продвинутого анализа нужен еще более обширный массив данных (например, по другим лигам, а не только по английской)
- Можно сильнее учитывать актуальные факторы, кроме Elo-rating, например, результаты недавних матчей, травмированных игроков, индивидуальный перфоманс игроков и тд
- Также можно попробовать обучить еще больше моделей (SVM, градиентный бустинг, ...)